In [ ]:
import os
from shutil import copyfile
from tqdm import tqdm
import csv
from datetime import datetime

In [ ]:
def flare_or_not(peak_flux):
    if peak_flux >= 1e-5:
        return True
    else:
        return False

In [ ]:
def read_csv(csv_path):
    meta_data = []
    first_time = True
    #path + 'meta_data.csv'

    with open(csv_path) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            if first_time:
                first_time = False
                continue
            meta_data.append([row[0], datetime.strptime(str(row[1])[:-3], '%Y-%m-%d %H:%M:%S.%f'), datetime.strptime(str(row[2])[:-3], '%Y-%m-%d %H:%M:%S.%f'), float(row[3]), flare_or_not(float(row[3]))])
    return meta_data

In [ ]:
def get_file_name_list(csv_path):
    meta_data = read_csv(csv_path)
    file_name_list = []
    for entry in meta_data:
        file_name_list.append([str(entry[1].year) + '-' + str('{:02d}'.format(entry[1].month)) + '-' + str('{:02d}'.format(entry[1].day)) + 'T' + str('{:02d}'.format(entry[1].hour)) + str('{:02d}'.format(entry[1].minute)) + str('{:02d}'.format(entry[1].second)) + '__magnetogram.jpg', entry[4]])
        
    return file_name_list

In [7]:
paths = ['../sdobenchmark_full/test/', '../sdobenchmark_full/training/']

flare_counter = 0
no_flare_counter = 0

for path in paths:
    magnetogram_file_names = get_file_name_list(path + 'meta_data.csv')
    for root,d_names,f_names in tqdm(os.walk(path)):
        for file in f_names:
            if 'magnetogram' in file and [file, True] in magnetogram_file_names:
                copyfile(root + '/' + file, './dataset/flare/' + str(flare_counter) + '.' + file.split('.')[-1])
                flare_counter = flare_counter + 1
            elif 'magnetogram' in file and [file, False] in magnetogram_file_names:
                copyfile(root + '/' + file, './dataset/no_flare/' + str(no_flare_counter) + '.' + file.split('.')[-1])
                if no_flare_counter == 681:
                    break
                no_flare_counter = no_flare_counter + 1

979it [00:00, 3408.57it/s]
9429it [00:11, 826.59it/s]
